# Distance matrices

In this notebook, we will  create the distance matrices. We will have to matrices: one for gas cars ane electric cars and the other one for drones. The distance matrices will be used to calculate the cost of the routes.

Each index will be a coordinate in the grid. The distance between two coordinates will be the Havesian distance between them (in the case of drones) and the one given by OSRM (in the case of vehicles).

In [ ]:
import pandas as pd

# Data for Gas Cars
gas_cars = {
    "Vehicle": "Gas Car",
    "Freight Rate [COP/km]": 5000,
    "Time Rate [COP/min]": 500,
    "Daily Maintenance [COP/day]": 30000,
    "Recharge/Fuel Cost [COP/(gal or kWh)]": 16000,
    "Recharge/Fuel Time [min/10 percent charge]": 0.1,
    "Avg. Speed [km/h]": None,
    "Gas Efficiency [km/gal]": 10,
}

# Data for Drones
drones = {
    "Vehicle": "Drone",
    "Freight Rate [COP/km]": 500,
    "Time Rate [COP/min]": 500,
    "Daily Maintenance [COP/day]": 3000,
    "Recharge/Fuel Cost [COP/(gal or kWh)]": 220.73,
    "Recharge/Fuel Time [min/10 percent charge]": 2,
    "Avg. Speed [km/h]": 40,
    "Gas Efficiency [km/gal]": None,
    "Electricity Efficency [kWh/km]": 0.15,
}


# Data for Solar EVs
solar_ev = {
    "Vehicle": "Solar EV",
    "Freight Rate [COP/km]": 4000,
    "Time Rate [COP/min]": 500,
    "Daily Maintenance [COP/day]": 21000,
    "Recharge/Fuel Cost [COP/(gal or kWh)]": None,
    "Recharge/Fuel Time [min/10 percent charge]": None,
    "Avg. Speed [km/h]": None,
    "Gas Efficiency [km/gal]": None,
    "Electricity Efficency [kWh/km]": 0.15,
}

# Combine into a dataframe
vehicles_data = pd.DataFrame([gas_cars, drones, solar_ev])


,Vehicle,Freight Rate [COP/km],Time Rate [COP/min],Daily Maintenance [COP/day],Recharge/Fuel Cost [COP/(gal or kWh)],Recharge/Fuel Time [min/10 percent charge],Avg. Speed [km/h],Gas Efficiency [km/gal],Electricity Efficency [kWh/km]
0,Gas Car,5000,500,30000,16000.00,0.1,NaN,10.0,NaN
1,Drone,500,500,3000,220.73,2.0,40.0,NaN,0.15
2,Solar EV,4000,500,21000,NaN,NaN,NaN,NaN,0.15


In [1]:
C_load_per_min = 500 # [COP/min] Every minute spent at loading/unloading products form the vehicle independently of the vehicle type
loading_speed_5kg_per_min = 1 # [kg/min] Loading speed for 5kg of product

loading_data = {
    "Activity": ["Loading/Unloading"],
    "Cost [COP/min]": [C_load_per_min],
    "Loading Speed [kg/min]": [loading_speed_5kg_per_min]
}


In [ ]:
from __future__ import division
from pyomo.environ import *
import pandas  as pd
from pyomo.opt import SolverFactory
import networkx as nx
from matplotlib import pyplot as plt

class SensorPlacementModel:
    def __init__(self, gas_cars, drones, solar_ev, car_matrix_distance, car_matrix_time,drone_matrix, clients, depots, nodes, num_depots, num_clientes, num_cars, cars):
        """
        Initialize the model parameters.
        """
        self.gas_cars = gas_cars
        self.drones = drones
        self.solar_ev = solar_ev
        self.car_matrix_distance = car_matrix_distance
        self.car_matrix_time = car_matrix_time
        self.drone_matrix = drone_matrix
        self.clients = clients
        self.depots = depots
        self.nodos = nodes
        self.num_clients = num_clientes
        self.num_depots = num_depots
        self.num_cars = num_cars
        self.cars = cars


        # Create the Pyomo model
        self.model = ConcreteModel()

    def build_model(self):
        """
        Build the optimization model.
        """
        model = self.model

        # Sets
        model.depots_set = RangeSet(0, self.num_depots-1) #Set of depots. In the cost_matrix, the depots come first  
        model.clients_set = RangeSet(self.num_depots, self.num_clients+self.num_depots-1) #Set of clients. In the cost matrix, the clients come after the depots.
        model.nodes = RangeSet(0, self.car_matrix.size -1) #Set of nodes. This includes both depots and clients.
        model.cars_set = RangeSet(0, self.num_cars-1) #Set of vehicles.
        
        #Helper Sets
        model.U = RangeSet(1, len(model.nodes)-1) #For the weird condition


        # Decision Variables
        model.x = Var(model.nodes, model.nodes, model.cars_set, domain=Binary) #Decision variables specifying if a given car goes from node i to node j
        model.c = Var(model.cars_set, domain=NonNegativeIntegers)  # Total load given to each car k
        model.u = Var(model.nodes, model.cars_set, domain=NonNegativeReals, bounds = (0, len(model.nodes)-1))


        # Objective: Minimize total cost (energy, installation, and communication costs)
        def obj_expression(model):
            #Cost of loading a car
            cost1 = sum(model.c[k]*500 for k in model.cars_set)

            #Cost of driving the car (distance)
            cost2_1 = sum(self.car_matrix_distance[i][j]*model.x[i,j,k]*self.cars[k]["Price_per_km"] for i in model.nodes for j in model.nodes for k in model.cars_set if k not in self.drones) #Rev, but u got the idea
            cost2_2 = sum(self.drone_matrix[i][j]*model.x[i,j,k]*self.cars[k]["Price_per_km"] for i in model.nodes for j in model.nodes for k in model.cars_set if k in self.drones) #Rev, nut u got the idea.

            #Cost of driving the car (time)
            cost3_1 = sum(self.car_matrix_time[i][j]*model.x[i,j,k]*self.cars[k]["Price_per_hour"] for i in model.nodes for j in model.nodes for k in model.cars_set if k not in self.drones) #Rev, but u got the idea
            cost3_2 = sum(self.drone_matrix[i][j]*40*model.x[i,j,k]*self.cars[k]["Price_per_hour"] for i in model.nodes for j in model.nodes for k in model.cars_set if k in self.drones) #Rev, nut u got the idea.

            #Cost for gas
            cost4 = sum((k_range[k]*recharge_fuel_cost[k]/efficiency[k]) for k in model.cars_set) #Rev, but u got the idea

            #Cost for maintenance
            cost5 = sum(k_maintenance[k] for k in model.cars_set)

            return cost1 + cost2_1 + cost2_2 + cost3_1 + cost3_2 + cost4 + cost5

        model.obj = Objective(rule=obj_expression, sense=minimize)

        # Constraints: Full coverage for each location with connection restriction
        model.full_coverage = ConstraintList()
        for l in model.locations:
            for s in model.sensor_types:
                if self.coverage_matrix[s, l] == 1:  # Sensor can cover location
                    # Ensure that each location is covered, but only if it is connected via the adjacency matrix
                    model.full_coverage.add(
                        sum(model.x[s, loc] for loc in model.locations if self.coverage_matrix[s, loc] == 1 and (loc, l) in self.adjacency_matrix and self.adjacency_matrix[loc, l] == 1) >= 1
                    )

        # Consistency Constraint: Total number of sensors of type s must match the placement decisions
        def sensor_consistency_rule(model, s):
            return model.y[s] == sum(model.x[s, l] for l in model.locations)

        model.sensor_consistency = Constraint(model.sensor_types, rule=sensor_consistency_rule)

        return model

    def solve_model(self):
        """
        Solve the model using the given solver.
        """
        solver = pyo.SolverFactory('highs' )
        results = solver.solve(self.model)
        return results

    def display_results(self):
        """
        Display the results of the optimization.
        """
        self.model.display()
    def print_output(self):
        """
        Plot sensor placement solutions on directed graphs using NetworkX for each sensor type in a subplot.
        """
        # Assign colors to sensor types
        sensor_type_colors = {
            self.sensor_types[0]: 'red',
            self.sensor_types[1]: 'green',
            self.sensor_types[2]: 'blue'
        }

        # Create subplots: 1 row, 3 columns (one for each sensor type)
        fig, axes = plt.subplots(1, 3, figsize=(18, 6))
        
        # Iterate over each sensor type
        for idx, s in enumerate(self.sensor_types):
            G = nx.DiGraph()  # Create a new directed graph for each sensor type

            # Add directed edges based on adjacency and sensor coverage for sensor type 's'
            for loc1 in self.locations:
                for loc2 in self.locations:
                    # Add a directed edge if the adjacency matrix indicates they are connected and covered by the sensor
                    if loc1 != loc2 and self.adjacency_matrix[loc1, loc2] == 1:
                        # Check if sensor type 's' is installed in loc1, and mark the edge color
                        if self.model.x[s, loc1]() == 1 and self.coverage_matrix[s, loc2] == 1:
                            G.add_edge(loc1, loc2, color=sensor_type_colors[s])  # Use sensor type color
                        else:
                            G.add_edge(loc1, loc2, color='black')  # Uncolored edges as black

            # Create node color mapping based on the decision variable
            node_colors = []
            for loc in G.nodes():
                if self.model.x[s, loc]() == 1:
                    node_colors.append(sensor_type_colors[s])  # Color nodes where this sensor is installed
                else:
                    node_colors.append('lightgray')  # Gray for nodes where this sensor is not installed

            # Get edge colors from the graph attributes
            edge_colors = [G[u][v]['color'] for u, v in G.edges]

            # Generate positions for nodes
            pos = nx.spring_layout(G)

            # Draw the directed graph with node and edge colors in the current axis
            nx.draw(G, pos, ax=axes[idx], node_color=node_colors, edge_color=edge_colors, with_labels=True, node_size=500, font_weight='bold', arrows=True)

            # Set the title for this subplot
            axes[idx].set_title(f"Sensor Type: {s}", fontsize=15)

        # Adjust layout to prevent overlapping
        plt.tight_layout()
        
        # Show the combined plot
        plt.show()